In [2]:
import time, sys, os
import h5py
import numpy as np
import scipy
from scipy import stats
import astropy
from astropy.cosmology import WMAP9 as cosmo
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import fsps
import sedpy
import prospect
import emcee
import dynesty
from prospect.models import priors
from prospect.models.templates import TemplateLibrary
from prospect.likelihood import lnlike_spec, lnlike_phot, write_log
from prospect.likelihood import chi_spec, chi_phot
from prospect.fitting import lnprobfn
from prospect.fitting import fit_model
from prospect.io import write_results as writer
from prospect import prospect_args
import prospect.io.read_results as reader
from multiprocessing import Pool
import subprocess
import pickle
home_dir = '/Users/brian/Library/CloudStorage/OneDrive-UniversityofHertfordshire/DwarfAGN'
cl_dir = '/beegfs/car/bbichanga/DwarfAGN'

dir_pick = [home_dir,cl_dir]
dir_pick = dir_pick[0]

cosmos = astropy.io.fits.open(dir_pick+'/COSMOS2020_FARMER_R1_v2.1_p3.fits',memmap=True)

In [3]:
cosmos_dat = cosmos[1].data

In [4]:
dire = '/Users/brian/Library/CloudStorage/OneDrive-UniversityofHertfordshire/DwarfAGN/dwarf_agn/emcee1/' # my data


In [27]:
dwarf_picker = np.where((cosmos_dat['lp_zPDF_l68'] > 0.1) & (cosmos_dat['lp_zPDF_u68'] < 0.25) & (cosmos_dat['lp_zPDF_l68'] > 0.1) & (cosmos_dat['lp_mass_med_max68'] < 9.5)
                        & (cosmos_dat['lp_mass_med_min68'] > 8.))[0]

In [28]:
cosmos_dat['ID'][dwarf_picker][3103]

774350

In [29]:
import prospect.io.read_results as reader
import numpy as np
from scipy.special import gamma, gammainc
from prospect.plotting.corner import quantile
import re
results_type = "emcee"
import os

directory = os.fsencode(dire)
import pandas as pd
columns = ['mass', 'dust', 'tage', 'tau', 'fagn', 'agn_tau', 'duste_umin', 'duste_qpah', 'duste_gamma','mass16', 'dust16', 'tage16', 'tau16', 'fagn16', 'agn_tau16', 'duste_umin16', 'duste_qpah16', 'duste_gamma16', 'psi', 'psi1']
keey = []


for file in os.listdir(directory):
    filename = os.fsdecode(file)
    f  = [int(s) for s in filename.split('_') if s.isdigit()]
    # print(f, filename)
    try:
        f  = [int(s) for s in filename.split('_') if s.isdigit()]
        # print(f)
        new_string = re.sub('%s'%(f[0]), '%s' %(cosmos_dat['ID'][dwarf_picker][f][0]), filename)
        keey.append([f[0],cosmos_dat['ID'][dwarf_picker][f][0]])
    except:
        f  = [int(s) for s in filename if s.isdigit()]
        a = map(str, f)    
        b = ''.join(a) 
        # c =int(b)
        f = []
        f = f.append(b)
        new_string = re.sub('%s'%(f) , '%s' %(cosmos_dat['ID'][dwarf_picker][f][0]), filename)
    
    # os.rename(filename, new_string)
print(new_string)

gal_4638_post_pcts.npy


In [ ]:
import prospect.io.read_results as reader
import numpy as np
from scipy.special import gamma, gammainc
from prospect.plotting.corner import quantile
    
results_type = "emcee"
import os

directory = os.fsencode(dire)
import pandas as pd
columns = ['mass', 'dust', 'tage', 'tau', 'fagn', 'agn_tau', 'duste_umin', 'duste_qpah', 'duste_gamma','mass16', 'dust16', 'tage16', 'tau16', 'fagn16', 'agn_tau16', 'duste_umin16', 'duste_qpah16', 'duste_gamma16', 'psi', 'psi1']
a = []


for file in os.listdir(directory):
    filename = os.fsdecode(file)
    f  = [int(s) for s in filename.split('_') if s.isdigit()]
    if filename.endswith(".h5") and len(filename)>23 and f[0] in arr: 
        try:
            result, obs, model  = reader.results_from(os.path.join(dire, filename), dangerous=False)

            # Maximum posterior probability sample
            imax = np.argmax(result['lnprobability'])
            csz = result["chain"].shape
            # if result["chain"].ndim > 2:
            # emcee
            i, j = np.unravel_index(imax, result['lnprobability'].shape)
            theta_max = result['chain'][i, j, :].copy()
            flatchain = result["chain"].reshape(csz[0] * csz[1], csz[2])
            #         else:
            #     # dynesty
            # theta_max = result['chain'][imax, :].copy()
            # flatchain = result["chain"]

            # 16th, 50th, and 84th percentiles of the posterior

            weights = result.get("weights", None)

            post_pcts = np.median(flatchain.T,axis=1)
            post_pcts16 = quantile(flatchain.T, q=[0.16, 0.50, 0.84], weights=weights)

            #----------------------------------------------------------------------------------------
            mass, dust, tage, tau, fagn, agn_tau, duste_umin, duste_qpah, duste_gamma = post_pcts[0],post_pcts[1],post_pcts[2],post_pcts[3],post_pcts[4],post_pcts[5],post_pcts[6],post_pcts[7],post_pcts[8]
            mass16, dust16, tage16, tau16, fagn16, agn_tau16, duste_umin16, duste_qpah16, duste_gamma16 = post_pcts16[0,0],post_pcts16[1,0],post_pcts16[2,0],post_pcts16[3,0],post_pcts16[4,0],post_pcts16[5,0],post_pcts16[6,0],post_pcts16[7,0],post_pcts16[8,0]
            #----------------------------------------------------------------------------------------
            #We calculate the star formation rate:

            tage, tau, mass = post_pcts[2], post_pcts[3], post_pcts[0]
            # for delay tau this function gives the (unnormalized) SFR 
            # for any t, tau combo in M_sun/Gyr
            # sfr = lambda t,tau: return (t/tau) * np.exp(-t/tau)

            sfr = lambda t,tau:(t/tau) * np.exp(-t/tau)
            # now we numerically integrate this SFH from 0 to tage to get the mass formed
            times = np.linspace(0, tage, 1000)
            A = np.trapz(sfr(times, tau), times)
            # But this could also be done using an incomplete gamma function (integral of xe^{-x})
            A = tau * gamma(2) * gammainc(2, tage/tau)
            # and now we renormalize the formed mass to the actual mass value 
            # to get the the SFR in M_sun per Gyr 
            psi = mass * sfr(tage, tau) / A
            # if we want SFR in Msun/year
            psi1 = psi/1e9

            row = mass, dust, tage, tau, fagn, agn_tau, duste_umin, duste_qpah, duste_gamma,mass16, dust16, tage16, tau16, fagn16, agn_tau16, duste_umin16, duste_qpah16, duste_gamma16, psi, psi1,
            a.append(row)
        except:
            print(filename)
            
    
df = pd.DataFrame(a,columns=columns)

